### #Step 1 - Hello,Data!
Load raw CSV (using only 500rows), display first 3 rows

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/1000 Sales Records.csv", nrows=500)
print(df.shape)
print(df.dtypes)
df.head(3)

(500, 14)
Region             object
Country            object
Item Type          object
Sales Channel      object
Order Priority     object
Order Date         object
Order ID            int64
Ship Date          object
Units Sold          int64
Unit Price        float64
Unit Cost         float64
Total Revenue     float64
Total Cost        float64
Total Profit      float64
dtype: object


,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Middle East and North Africa,Libya,Cosmetics,Offline,M,10/18/2014,686800706,10/31/2014,8446,437.20,263.33,3692591.20,2224085.18,1468506.02
1,North America,Canada,Vegetables,Online,M,11/7/2011,185941302,12/8/2011,3018,154.06,90.93,464953.08,274426.74,190526.34
2,Middle East and North Africa,Libya,Baby Food,Offline,C,10/31/2016,246222341,12/9/2016,1517,255.28,159.42,387259.76,241840.14,145419.62


### #Step 2 - Pick the Right Container
Justify dict vs namedtuple vs sets(1–2 sentences)  

- dict = Data type that stores {key : value} pairs. Used for data retrieval and management.  
- set = Data type that stores a set of unique values that do not overlap.  
- namedtuple = Create an immutable object that can access data by name. Keep each line as a record, it's easy to read.

### #Step 3 - Implement Functions and  Data structure
Implement and use it to populate an data structure

In [2]:
from collections import namedtuple

Sale = namedtuple("Sale", "date, customer_id, product, price, quantity, coupon_code, shipping_city, country")

# https://www.freecodecamp.org/news/python-namedtuple-examples-how-to-create-and-work-with-namedtuples/ - namedtuple
#Sale._fields

def build_records(df: pd.DataFrame):
    records = []
    for _, r in df.iterrows():           # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iterrows.html  - iterrows
        records.append(                  # https://wikidocs.net/84389 - study with records
            Sale(
                date= r["Order Date"],
                customer_id= r["Order ID"], 
                product= r["Item Type"],
                price= float(r["Unit Price"]),
                quantity= int(r["Units Sold"]),
                coupon_code= None,        # No Data
                shipping_city= None,      # No Data
                country= r["Country"],
            )
        )
    return records

records = build_records(df)    #  populate
records[0:2]                   # Check 2rows


[Sale(date='10/18/2014', customer_id=686800706, product='Cosmetics', price=437.2, quantity=8446, coupon_code=None, shipping_city=None, country='Libya'),
 Sale(date='11/7/2011', customer_id=185941302, product='Vegetables', price=154.06, quantity=3018, coupon_code=None, shipping_city=None, country='Canada')]

### #Step 4 - Bulk Loaded
Example: Map data structures from dataframes to dictionaries

In [3]:
# Total Revenue = Unit Price * Units Sold
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_index.html

Order_info = (
    df.set_index("Order ID")[["Order Date", "Unit Price", "Units Sold", "Country"]].to_dict(orient="index") # Key : index(Order ID) / Value: each columns' values
)

print(Order_info[246222341])

revenue_by_country = (
    df.groupby("Country")["Total Revenue"].sum().to_dict() # Key : Country /  Value : Sum of Total Revenue of each Country     
)
Countryname = "Canada"
print("Sum of Total Revenue of", Countryname, " : ",revenue_by_country[Countryname])  # example: Check Canadian total sales

{'Order Date': '10/31/2016', 'Unit Price': 255.28, 'Units Sold': 1517, 'Country': 'Libya'}
Sum of Total Revenue of Canada  :  1226103.3


### #Step 5 - Quick Profiling
Min/mean/max price, unique city count (set)
 - Datatype is already numeric (if it is not numeric, change datatype -> df["Unit Price"] = pd.to_numeric(df["Unit Price"])

In [4]:
price_min = df["Unit Price"].min()
price_mean = df["Unit Price"].mean()
price_max = df["Unit Price"].max()

unique_city_count = len(set(df["Country"]))

print("Min",price_min)
print("Mean",price_mean)
print("Max",price_max)
print("City Count",unique_city_count)

Min 9.33
Mean 274.29506
Max 668.27
City Count 171


### #Step 6 - Spot the Grime
Identify at least three dirty data cases

In [5]:
# https://pandas.pydata.org/docs/reference/api/pandas.Series.str.strip.html
# https://pandas.pydata.org/docs/reference/api/pandas.Series.str.title.html
df["Order Date"] = pd.to_datetime(df["Order Date"])
df["Ship Date"]  = pd.to_datetime(df["Ship Date"])

# (1) Ship Date <= Order Date
dirty_ship_before_order = df["Ship Date"] <= df["Order Date"]

# (2) Region - Messy text
dirty_messy_region = df["Region"].astype(str) != df["Region"].astype(str).str.strip()

# (3) Country - Messy text
dirty_messy_country = (df["Country"].astype(str) != df["Country"].astype(str).str.strip())

# Print Dirty data case
# print("missing_dates :", (dirty_missing_dates.sum()))
print("ship_before_order :", (dirty_ship_before_order.sum()))
print("Region_messy_text : ", (dirty_messy_region.sum()))
print("Country_messy_text :", (dirty_messy_country.sum()))

#df.loc[dirty_messy_country, ["Country"]].head(10)
df.loc[dirty_ship_before_order, ["Ship Date","Order Date"]].head(10)

ship_before_order : 8
Region_messy_text :  0
Country_messy_text : 22


,Ship Date,Order Date
33,2014-04-08,2014-04-08
88,2011-09-04,2011-09-04
119,2010-07-18,2010-07-18
183,2016-05-15,2016-05-15
294,2017-02-25,2017-02-25
348,2013-04-18,2013-04-18
472,2014-09-21,2014-09-21
498,2015-02-22,2015-02-22


### #Step 7 - Cleaning Rules
Execute fixes inside clean(); show “before/after” counts

In [6]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html - using .loc to delete row if boolean is false

class clean_total:
    def clean(df: pd.DataFrame):
        cd = df.copy()

        messy_ship = (cd["Ship Date"] <= cd["Order Date"])
        messy_country = (cd["Country"].astype(str) != cd["Country"].astype(str).str.strip())

        before = {
            "Rows" : len(cd),
            "Ship_before_same" : int(messy_ship.sum()),
            "Country_Messy" : int(messy_country.sum())
        }

        # Cleaning
        cd = cd.loc[cd["Ship Date"] > cd["Order Date"]].copy()
        cd["Country"] = cd["Country"].astype("string").str.strip()

        messy_ship_a = (cd["Ship Date"] <= cd["Order Date"])
        messy_country_a = (cd["Country"].astype(str) != cd["Country"].astype(str).str.strip())

        after = {
            "Rows" : len(cd),
            "Ship_before_same" : int(messy_ship_a.sum()),
            "Country_Messy" : int(messy_country_a.sum())
        }

        return cd, before, after

    def total(df: pd.DataFrame) :
        cd = df.copy()
        total_sum = float((cd["Unit Price"] * cd["Units Sold"]).sum())
        return cd, total_sum
    
cd, before, after = clean_total.clean(df)
print("Before : ", before)
print("After  : ", after)
print(f"Rows: {before['Rows']} -> {after['Rows']}")

Before :  {'Rows': 500, 'Ship_before_same': 8, 'Country_Messy': 22}
After  :  {'Rows': 492, 'Ship_before_same': 0, 'Country_Messy': 0}
Rows: 500 -> 492


### #Step 8 - Transformations
For example: Parse coupon_code ➞ numeric discount (others apply)

In [7]:
cl_df = cd.copy()

cl_df["coupon_code"] = "no coupon"
for x, val in cl_df.iterrows(): 
    onoff = str(val["Sales Channel"].strip().lower())
    if onoff == "online":
        cl_df.at[x,"coupon_code"] = "onlinecoupon_10%"
    elif onoff == "offline":
        cl_df.at[x,"coupon_code"] = "offlinecoupon_5%"

print(cl_df[["Sales Channel", "coupon_code"]].head(3))

discount_rates=[]
for code in cl_df["coupon_code"]:
    if code == "onlinecoupon_10%":
        discount_rates.append(0.1)
    elif code == "offlinecoupon_5%":
        discount_rates.append(0.05)
    else :
        discount_rates.append(0)

cl_df["discount_rates"] = discount_rates

print (cl_df["discount_rates"].head(3))


  Sales Channel       coupon_code
0       Offline  offlinecoupon_5%
1        Online  onlinecoupon_10%
2       Offline  offlinecoupon_5%
0    0.05
1    0.10
2    0.05
Name: discount_rates, dtype: float64


### #Step 9 - Feature Engineering
For example: Add days_since_purchase

In [8]:
# https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.normalize.html
today = pd.Timestamp("today").normalize()

# After Purchase
cl_df["days_since_purchase"] = (today - cl_df["Order Date"]).dt.days
# After Ship
cl_df["days_since_ship"] = (today - cl_df["Ship Date"]).dt.days

print(cl_df[["Order Date","days_since_purchase","Ship Date","days_since_ship"]].head(10))

  Order Date  days_since_purchase  Ship Date  days_since_ship
0 2014-10-18                 3999 2014-10-31             3986
1 2011-11-07                 5075 2011-12-08             5044
2 2016-10-31                 3255 2016-12-09             3216
3 2010-04-10                 5651 2010-05-12             5619
4 2011-08-16                 5158 2011-08-31             5143
5 2014-11-24                 3962 2014-12-28             3928
6 2015-03-04                 3862 2015-04-17             3818
7 2012-05-17                 4883 2012-06-28             4841
8 2015-01-29                 3896 2015-03-07             3859
9 2013-12-24                 4297 2014-01-19             4271


### #Step 10 - Mini-Aggregation
For example: Revenue per shipping_city (dict or pandas.groupby)

In [9]:
main = cl_df.copy()

capital = pd.read_csv("data/concap.csv")
capital = capital.rename(columns={"CountryName": "Country", "CapitalName":"shipping_city"})
capital = capital[["Country", "shipping_city"]].copy()

capital["Country"]  = capital["Country"].astype("string").str.strip()
capital["shipping_city"] = capital["shipping_city"].astype("string").str.strip()  

cl_df = main.merge(capital, on= "Country", how="left")
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html

cl_df.head(3)

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit,coupon_code,discount_rates,days_since_purchase,days_since_ship,shipping_city
0,Middle East and North Africa,Libya,Cosmetics,Offline,M,2014-10-18,686800706,2014-10-31,8446,437.20,263.33,3692591.20,2224085.18,1468506.02,offlinecoupon_5%,0.05,3999,3986,Tripoli
1,North America,Canada,Vegetables,Online,M,2011-11-07,185941302,2011-12-08,3018,154.06,90.93,464953.08,274426.74,190526.34,onlinecoupon_10%,0.10,5075,5044,Ottawa
2,Middle East and North Africa,Libya,Baby Food,Offline,C,2016-10-31,246222341,2016-12-09,1517,255.28,159.42,387259.76,241840.14,145419.62,offlinecoupon_5%,0.05,3255,3216,Tripoli


In [10]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(cl_df):
    # Performed 1 aggregation grouped on column: 'shipping_city'
    cl_df = cl_df.groupby(['shipping_city']).agg(TotalRevenue_sum=('Total Revenue', 'sum')).reset_index()
    return cl_df

cl_df_clean = clean_data(cl_df.copy())
cl_df_clean.head()

,shipping_city,TotalRevenue_sum
0,Abu Dhabi,665128.18
1,Abuja,131748.76
2,Accra,8138014.18
3,Addis Ababa,281251.62
4,Algiers,4668819.12


### #Step 11 - Serialization Checkpoint
Save cleaned data to JSON

In [11]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_json.html

cl_df.to_json("data/cleand_data.json",orient="records",indent=1)
cl_df.to_csv("data/cleand_data.csv")
cl_df_clean.to_json("data/city_aggregate.json",orient="records",indent=1)
cl_df_clean.to_csv("data/city_aggregate.csv")

### #Step 12 - Soft Interview Reflection
Markdown: < 120 words explaining how Functions have helped

I built a small class, clean_total, with clean() and total().  
clean() drops abnormal rows and trims whitespace in Country, and prints before/after counts to verify fixes.   
If the class built as a .py module and import it in the notebook, than I can reuse it by changing only parameters, improving readability.  
In Steps 9–11, I created derived columns, merged a secondary metadata file to add a city per Country, and used groupby for city-level aggregation.   
I implemented a basic pipeline based on the course material; with more advanced pandas/numpy features, the workflow can be made simpler and more reliable.  

### Data-Dictionary Section

In [12]:
print (cl_df.info())
cl_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492 entries, 0 to 491
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Region               492 non-null    object        
 1   Country              492 non-null    string        
 2   Item Type            492 non-null    object        
 3   Sales Channel        492 non-null    object        
 4   Order Priority       492 non-null    object        
 5   Order Date           492 non-null    datetime64[ns]
 6   Order ID             492 non-null    int64         
 7   Ship Date            492 non-null    datetime64[ns]
 8   Units Sold           492 non-null    int64         
 9   Unit Price           492 non-null    float64       
 10  Unit Cost            492 non-null    float64       
 11  Total Revenue        492 non-null    float64       
 12  Total Cost           492 non-null    float64       
 13  Total Profit         492 non-null  

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit,coupon_code,discount_rates,days_since_purchase,days_since_ship,shipping_city
0,Middle East and North Africa,Libya,Cosmetics,Offline,M,2014-10-18,686800706,2014-10-31,8446,437.20,263.33,3692591.20,2224085.18,1468506.02,offlinecoupon_5%,0.05,3999,3986,Tripoli
1,North America,Canada,Vegetables,Online,M,2011-11-07,185941302,2011-12-08,3018,154.06,90.93,464953.08,274426.74,190526.34,onlinecoupon_10%,0.10,5075,5044,Ottawa
2,Middle East and North Africa,Libya,Baby Food,Offline,C,2016-10-31,246222341,2016-12-09,1517,255.28,159.42,387259.76,241840.14,145419.62,offlinecoupon_5%,0.05,3255,3216,Tripoli
3,Asia,Japan,Cereal,Offline,C,2010-04-10,161442649,2010-05-12,3322,205.70,117.11,683335.40,389039.42,294295.98,offlinecoupon_5%,0.05,5651,5619,Tokyo
4,Sub-Saharan Africa,Chad,Fruits,Offline,H,2011-08-16,645713555,2011-08-31,9845,9.33,6.92,91853.85,68127.40,23726.45,offlinecoupon_5%,0.05,5158,5143,N'Djamena


| Field               | Type             | Description                                | Source                     |
|---------------------|------------------|--------------------------------------------|----------------------------|
| Region              | Object           | Sales region or geographic area            | Primary CSV                |
| Country             | String           | Country                                    | Primary CSV                |
| Item Type           | object           | Product Category                           | Primary CSV                |
| Sales Channel       | object           | where sale occurred (Online/Offline)       | Primary CSV                |
| Order Priority      | object           | Priority level (L/M/H/C)                   | Primary CSV                |
| Order Date          | datetime64[ns]   | when the order was created                 | Primary CSV                |
| Order ID            | int64            | Unique identifier                          | Primary CSV                |
| Ship Date           | datetime64[ns]   | when the order was shipped                 | Primary CSV                |
| Units Sold          | int64            | Quantity of units sold                     | Primary CSV                |
| Unit Price          | float64          | Price of Unit                              | Primary CSV                |
| Unit Cost           | float64          | Cost of Unit                               | Primary CSV                |
| Total Revenue       | float64          | Unit Price * Units Sold                    | Primary CSV                |
| Total Cost          | float64          | Unit Cost * Units Sold                     | Primary CSV                |
| Total Profit        | float64          | Total Revenue - Total Cost                 | Primary CSV                |
| coupon_code         | object           | Coupon code derived from Sales Channel     | Synthetic (rule) step8     |
| discount_rates      | float64          | Numeric discount mapped from coupon_code   | Derived (coupon_code)      |
| days_since_purchase | int64            | today - order date                         | Derived (calc)             |
| days_since_ship     | int64            | today - ship date                          | Derived (calc)             |
| shipping_city       | string           | Derived via Country -> Capital             | Secondary metadata file    |


Secondary metadata : https://www.kaggle.com/datasets/nikitagrec/world-capitals-gps